<a href="https://colab.research.google.com/github/marco-siino/gpt_energy_cost_optimization/blob/main/Source_code_from_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


### Codice Python Aggiornato

from datetime import datetime, timedelta

# Definiamo le fasce orarie con i relativi costi
fasce_orarie = {
    "F1": {"start": "08:00", "end": "19:00", "costo": 0.25},
    "F2": {"start": "07:00", "end": "08:00", "costo": 0.15},
    "F2_b": {"start": "19:00", "end": "23:00", "costo": 0.15},
    "F3": {"start": "23:00", "end": "07:00", "costo": 0.10}
}

# Definiamo i consumi degli elettrodomestici in kW
consumi = {
    "lavatrice": 1.0,
    "asciugatrice": 1.2,
    "lavastoviglie": 1.0
}

# Funzione per convertire stringhe di tempo in oggetti datetime
def str_to_datetime(date_str, time_str):
    return datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

# Funzione per calcolare la durata in ore tra due datetime
def calcola_durata(start, end):
    return (end - start).seconds / 3600

# Funzione per determinare la fascia oraria di un timestamp
def determina_fascia(timestamp):
    time = timestamp.time()
    for fascia, orari in fasce_orarie.items():
        start = datetime.strptime(orari["start"], "%H:%M").time()
        end = datetime.strptime(orari["end"], "%H:%M").time()
        if start <= time < end or (start > end and (time >= start or time < end)):
            return fascia
    return None

# Funzione per calcolare il costo di utilizzo di un elettrodomestico
def calcola_costo_utilizzo(usage, date, appliance):
    costo_totale = 0.0
    for start_time, end_time in usage:
        start = str_to_datetime(date, start_time)
        end = str_to_datetime(date, end_time)
        durata = calcola_durata(start, end)
        fascia = determina_fascia(start)
        costo = durata * fasce_orarie[fascia]["costo"] * consumi[appliance]
        costo_totale += costo
    return costo_totale

# Funzione per verificare se un nuovo intervallo può essere aggiunto senza superare il limite di consumo
def verifica_consumo(intervalli, nuovo_inizio, nuovo_fine, nuovo_consumo):
    for inizio, fine, consumo in intervalli:
        if (inizio < nuovo_fine and nuovo_inizio < fine):  # Intervalli si sovrappongono
            if consumo + nuovo_consumo > 1.5:
                return False
    return True

# Dati di esempio
date = "2014-07-02"

usage_tumble_dryer = [
    ("11:05:34", "11:07:04"),
    ("11:21:04", "11:23:20"),
    ("16:09:05", "16:09:20")
]

usage_washing_machine = [
    ("02:22:20", "02:22:25"),
    ("12:18:34", "12:58:05"),
    ("13:00:49", "13:01:04"),
    ("13:08:49", "13:09:04")
]

usage_dishwasher = []  # Lavastoviglie non utilizzata

# Calcolo dei costi
costo_tumble_dryer = calcola_costo_utilizzo(usage_tumble_dryer, date, "asciugatrice")
costo_washing_machine = calcola_costo_utilizzo(usage_washing_machine, date, "lavatrice")

print(f"Costo totale Asciugatrice: €{costo_tumble_dryer:.4f}")
print(f"Costo totale Lavatrice: €{costo_washing_machine:.4f}")

# Funzione per programmare gli elettrodomestici rispettando il limite di consumo
def programma_elettrodomestici(usage_list, appliance, intervalli):
    nuovo_intervalli = []
    for start_time, end_time in usage_list:
        start = str_to_datetime(date, start_time)
        end = str_to_datetime(date, end_time)
        if verifica_consumo(intervalli, start, end, consumi[appliance]):
            nuovo_intervalli.append((start, end, consumi[appliance]))
        else:
            # Trova una nuova fascia oraria che rispetti il limite di consumo
            for fascia, orari in fasce_orarie.items():
                fascia_start = datetime.strptime(f"{date} {orari['start']}", "%Y-%m-%d %H:%M")
                fascia_end = datetime.strptime(f"{date} {orari['end']}", "%Y-%m-%d %H:%M")
                if verifica_consumo(intervalli, fascia_start, fascia_end, consumi[appliance]):
                    nuovo_intervalli.append((fascia_start, fascia_end, consumi[appliance]))
                    break
    return nuovo_intervalli

# Programma gli elettrodomestici
intervalli_utilizzo = []
intervalli_utilizzo += programma_elettrodomestici(usage_tumble_dryer, "asciugatrice", intervalli_utilizzo)
intervalli_utilizzo += programma_elettrodomestici(usage_washing_machine, "lavatrice", intervalli_utilizzo)

print("Nuovi intervalli di utilizzo:")
for inizio, fine, consumo in intervalli_utilizzo:
    print(f"Inizio: {inizio.strftime('%H:%M:%S')}, Fine: {fine.strftime('%H:%M:%S')}, Consumo: {consumo} kW")



Costo totale Asciugatrice: €0.0201
Costo totale Lavatrice: €0.1669
Nuovi intervalli di utilizzo:
Inizio: 11:05:34, Fine: 11:07:04, Consumo: 1.2 kW
Inizio: 11:21:04, Fine: 11:23:20, Consumo: 1.2 kW
Inizio: 16:09:05, Fine: 16:09:20, Consumo: 1.2 kW
Inizio: 02:22:20, Fine: 02:22:25, Consumo: 1.0 kW
Inizio: 12:18:34, Fine: 12:58:05, Consumo: 1.0 kW
Inizio: 13:00:49, Fine: 13:01:04, Consumo: 1.0 kW
Inizio: 13:08:49, Fine: 13:09:04, Consumo: 1.0 kW


In [ ]:
from datetime import datetime, timedelta
import heapq

# Definiamo le fasce orarie con i relativi costi
fasce_orarie = {
    "F1": {"start": "08:00", "end": "19:00", "costo": 0.25},
    "F2": {"start": "07:00", "end": "08:00", "costo": 0.15},
    "F2_b": {"start": "19:00", "end": "23:00", "costo": 0.15},
    "F3": {"start": "23:00", "end": "07:00", "costo": 0.10}
}

# Definiamo i consumi degli elettrodomestici in kW
consumi = {
    "lavatrice": 1.0,
    "asciugatrice": 1.2,
    "lavastoviglie": 1.0
}

# Funzione per convertire stringhe di tempo in oggetti datetime
def str_to_datetime(date_str, time_str):
    return datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

# Funzione per calcolare la durata in ore tra due datetime
def calcola_durata(start, end):
    return (end - start).seconds / 3600

# Funzione per determinare la fascia oraria di un timestamp
def determina_fascia(timestamp):
    time = timestamp.time()
    for fascia, orari in fasce_orarie.items():
        start = datetime.strptime(orari["start"], "%H:%M").time()
        end = datetime.strptime(orari["end"], "%H:%M").time()
        if start <= time < end or (start > end and (time >= start or time < end)):
            return fascia
    return None

# Funzione per calcolare il costo di utilizzo di un elettrodomestico
def calcola_costo_utilizzo(usage, date, appliance):
    costo_totale = 0.0
    for start_time, end_time in usage:
        start = str_to_datetime(date, start_time)
        end = str_to_datetime(date, end_time)
        durata = calcola_durata(start, end)
        fascia = determina_fascia(start)
        costo = durata * fasce_orarie[fascia]["costo"] * consumi[appliance]
        costo_totale += costo
    return costo_totale

# Funzione per verificare se un nuovo intervallo può essere aggiunto senza superare il limite di consumo
def verifica_consumo(intervalli, nuovo_inizio, nuovo_fine, nuovo_consumo):
    for inizio, fine, consumo in intervalli:
        if (inizio < nuovo_fine and nuovo_inizio < fine):  # Intervalli si sovrappongono
            if consumo + nuovo_consumo > 1.5:
                return False
    return True

# Dati di esempio
date = "2014-07-02"

usage_tumble_dryer = [
    ("11:05:34", "11:07:04"),
    ("11:21:04", "11:23:20"),
    ("16:09:05", "16:09:20")
]

usage_washing_machine = [
    ("02:22:20", "02:22:25"),
    ("12:18:34", "12:58:05"),
    ("13:00:49", "13:01:04"),
    ("13:08:49", "13:09:04")
]

usage_dishwasher = []  # Lavastoviglie non utilizzata

# Funzione per calcolare il costo di utilizzo di un intervallo
def calcola_costo_intervallo(start, end, appliance):
    durata = calcola_durata(start, end)
    fascia = determina_fascia(start)
    costo = durata * fasce_orarie[fascia]["costo"] * consumi[appliance]
    return costo

# Funzione di branch and bound per ottimizzare lo scheduling
def branch_and_bound(usage_list, appliance, intervalli, limite_consumo):
    queue = []
    heapq.heappush(queue, (0, [], 0))

    best_solution = None
    best_cost = float('inf')

    while queue:
        current_cost, current_schedule, current_consumo = heapq.heappop(queue)

        if current_cost >= best_cost:
            continue

        if len(current_schedule) == len(usage_list):
            if current_cost < best_cost:
                best_cost = current_cost
                best_solution = current_schedule
            continue

        for start_time, end_time in usage_list:
            start = str_to_datetime(date, start_time)
            end = str_to_datetime(date, end_time)
            if verifica_consumo(current_schedule, start, end, consumi[appliance]):
                new_cost = current_cost + calcola_costo_intervallo(start, end, appliance)
                new_consumo = current_consumo + consumi[appliance]
                new_schedule = current_schedule + [(start, end, consumi[appliance])]
                heapq.heappush(queue, (new_cost, new_schedule, new_consumo))

    return best_solution, best_cost

# Calcolo dei costi originali
costo_tumble_dryer = calcola_costo_utilizzo(usage_tumble_dryer, date, "asciugatrice")
costo_washing_machine = calcola_costo_utilizzo(usage_washing_machine, date, "lavatrice")

print(f"Costo totale Asciugatrice: €{costo_tumble_dryer:.4f}")
print(f"Costo totale Lavatrice: €{costo_washing_machine:.4f}")

# Programma gli elettrodomestici rispettando il limite di consumo
intervalli_utilizzo = []
intervalli_tumble_dryer, costo_tumble_dryer_ottimizzato = branch_and_bound(usage_tumble_dryer, "asciugatrice", intervalli_utilizzo, 1.5)
intervalli_utilizzo += intervalli_tumble_dryer

intervalli_washing_machine, costo_washing_machine_ottimizzato = branch_and_bound(usage_washing_machine, "lavatrice", intervalli_utilizzo, 1.5)
intervalli_utilizzo += intervalli_washing_machine

print("Nuovi intervalli di utilizzo:")
for inizio, fine, consumo in intervalli_utilizzo:
    print(f"Inizio: {inizio.strftime('%H:%M:%S')}, Fine: {fine.strftime('%H:%M:%S')}, Consumo: {consumo} kW")

print(f"Costo ottimizzato totale Asciugatrice: €{costo_tumble_dryer_ottimizzato:.4f}")
print(f"Costo ottimizzato totale Lavatrice: €{costo_washing_machine_ottimizzato:.4f}")


Costo totale Asciugatrice: €0.0201
Costo totale Lavatrice: €0.1669
Nuovi intervalli di utilizzo:
Inizio: 11:05:34, Fine: 11:07:04, Consumo: 1.2 kW
Inizio: 16:09:05, Fine: 16:09:20, Consumo: 1.2 kW
Inizio: 11:21:04, Fine: 11:23:20, Consumo: 1.2 kW
Inizio: 02:22:20, Fine: 02:22:25, Consumo: 1.0 kW
Inizio: 12:18:34, Fine: 12:58:05, Consumo: 1.0 kW
Inizio: 13:00:49, Fine: 13:01:04, Consumo: 1.0 kW
Inizio: 13:08:49, Fine: 13:09:04, Consumo: 1.0 kW
Costo ottimizzato totale Asciugatrice: €0.0201
Costo ottimizzato totale Lavatrice: €0.1669


In [ ]:
from datetime import datetime, timedelta
import heapq

# Definiamo le fasce orarie con i relativi costi
fasce_orarie = {
    "F1": {"start": "08:00", "end": "19:00", "costo": 0.25},
    "F2": {"start": "07:00", "end": "08:00", "costo": 0.15},
    "F2_b": {"start": "19:00", "end": "23:00", "costo": 0.15},
    "F3": {"start": "23:00", "end": "07:00", "costo": 0.10}
}

# Definiamo i consumi degli elettrodomestici in kW
consumi = {
    "lavatrice": 1.0,
    "asciugatrice": 1.2,
    "lavastoviglie": 1.0
}

# Funzione per convertire stringhe di tempo in oggetti datetime
def str_to_datetime(date_str, time_str):
    return datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

# Funzione per calcolare la durata in ore tra due datetime
def calcola_durata(start, end):
    return (end - start).seconds / 3600

# Funzione per determinare la fascia oraria di un timestamp
def determina_fascia(timestamp):
    time = timestamp.time()
    for fascia, orari in fasce_orarie.items():
        start = datetime.strptime(orari["start"], "%H:%M").time()
        end = datetime.strptime(orari["end"], "%H:%M").time()
        if start <= time < end or (start > end and (time >= start or time < end)):
            return fascia
    return None

# Funzione per calcolare il costo di utilizzo di un elettrodomestico
def calcola_costo_intervallo(start, end, appliance):
    durata = calcola_durata(start, end)
    fascia = determina_fascia(start)
    costo = durata * fasce_orarie[fascia]["costo"] * consumi[appliance]
    return costo

# Funzione per verificare se un nuovo intervallo può essere aggiunto senza superare il limite di consumo
def verifica_consumo(intervalli, nuovo_inizio, nuovo_fine, nuovo_consumo):
    for inizio, fine, consumo in intervalli:
        if (inizio < nuovo_fine and nuovo_inizio < fine):  # Intervalli si sovrappongono
            if consumo + nuovo_consumo > 1.5:
                return False
    return True

# Funzione di branch and bound per ottimizzare lo scheduling
def branch_and_bound(usage_list, appliance, intervalli, limite_consumo):
    best_solution = None
    best_cost = float('inf')

    def recursive_scheduling(schedule, current_cost, idx):
        nonlocal best_solution, best_cost

        if idx == len(usage_list):
            if current_cost < best_cost:
                best_cost = current_cost
                best_solution = schedule.copy()
            return

        start_time, end_time = usage_list[idx]
        start = str_to_datetime(date, start_time)
        end = str_to_datetime(date, end_time)
        durata = end - start

        for fascia, orari in fasce_orarie.items():
            fascia_start = datetime.strptime(f"{date} {orari['start']}", "%Y-%m-%d %H:%M")
            fascia_end = datetime.strptime(f"{date} {orari['end']}", "%Y-%m-%d %H:%M")

            current_start = max(fascia_start, start - (start - fascia_start) % timedelta(minutes=1))
            while current_start + durata <= fascia_end:
                current_end = current_start + durata
                if verifica_consumo(schedule, current_start, current_end, consumi[appliance]):
                    new_cost = current_cost + calcola_costo_intervallo(current_start, current_end, appliance)
                    schedule.append((current_start, current_end, consumi[appliance]))
                    recursive_scheduling(schedule, new_cost, idx + 1)
                    schedule.pop()
                current_start += timedelta(minutes=1)

    recursive_scheduling([], 0, 0)
    return best_solution, best_cost

# Dati di esempio
date = "2014-07-02"

usage_tumble_dryer = [
    ("11:05:34", "11:07:04"),
    ("11:21:04", "11:23:20"),
    ("16:09:05", "16:09:20")
]

usage_washing_machine = [
    ("02:22:20", "02:22:25"),
    ("12:18:34", "12:58:05"),
    ("13:00:49", "13:01:04"),
    ("13:08:49", "13:09:04")
]

usage_dishwasher = []  # Lavastoviglie non utilizzata

# Calcolo dei costi originali
costo_tumble_dryer = calcola_costo_utilizzo(usage_tumble_dryer, date, "asciugatrice")
costo_washing_machine = calcola_costo_utilizzo(usage_washing_machine, date, "lavatrice")

print(f"Costo totale Asciugatrice: €{costo_tumble_dryer:.4f}")
print(f"Costo totale Lavatrice: €{costo_washing_machine:.4f}")

# Programma gli elettrodomestici rispettando il limite di consumo
intervalli_utilizzo = []
intervalli_tumble_dryer, costo_tumble_dryer_ottimizzato = branch_and_bound(usage_tumble_dryer, "asciugatrice", intervalli_utilizzo, 1.5)
intervalli_utilizzo += intervalli_tumble_dryer

intervalli_washing_machine, costo_washing_machine_ottimizzato = branch_and_bound(usage_washing_machine, "lavatrice", intervalli_utilizzo, 1.5)
intervalli_utilizzo += intervalli_washing_machine

print("Nuovi intervalli di utilizzo:")
for inizio, fine, consumo in intervalli_utilizzo:
    print(f"Inizio: {inizio.strftime('%H:%M:%S')}, Fine: {fine.strftime('%H:%M:%S')}, Consumo: {consumo} kW")

print(f"Costo ottimizzato totale Asciugatrice: €{costo_tumble_dryer_ottimizzato:.4f}")
print(f"Costo ottimizzato totale Lavatrice: €{costo_washing_machine_ottimizzato:.4f}")


Costo totale Asciugatrice: €0.0201
Costo totale Lavatrice: €0.1669


KeyboardInterrupt: 

In [ ]:
from datetime import datetime, timedelta

# Definiamo le fasce orarie con i relativi costi
fasce_orarie = {
    "F1": {"start": "08:00", "end": "19:00", "costo": 0.25},
    "F2": {"start": "07:00", "end": "08:00", "costo": 0.15},
    "F2_b": {"start": "19:00", "end": "23:00", "costo": 0.15},
    "F3": {"start": "23:00", "end": "07:00", "costo": 0.10}
}

# Definiamo i consumi degli elettrodomestici in kW
consumi = {
    "lavatrice": 1.0,
    "asciugatrice": 1.2,
    "lavastoviglie": 1.0
}

# Funzione per convertire stringhe di tempo in oggetti datetime
def str_to_datetime(date_str, time_str):
    return datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

# Funzione per calcolare la durata in ore tra due datetime
def calcola_durata(start, end):
    return (end - start).seconds / 3600

# Funzione per determinare la fascia oraria di un timestamp
def determina_fascia(timestamp):
    time = timestamp.time()
    for fascia, orari in fasce_orarie.items():
        start = datetime.strptime(orari["start"], "%H:%M").time()
        end = datetime.strptime(orari["end"], "%H:%M").time()
        if start <= time < end or (start > end and (time >= start or time < end)):
            return fascia
    return None

# Funzione per calcolare il costo di utilizzo di un intervallo
def calcola_costo_intervallo(start, end, appliance):
    durata = calcola_durata(start, end)
    fascia = determina_fascia(start)
    costo = durata * fasce_orarie[fascia]["costo"] * consumi[appliance]
    return costo

# Funzione per verificare se un nuovo intervallo può essere aggiunto senza superare il limite di consumo
def verifica_consumo(intervalli, nuovo_inizio, nuovo_fine, nuovo_consumo):
    for inizio, fine, consumo in intervalli:
        if (inizio < nuovo_fine and nuovo_inizio < fine):  # Intervalli si sovrappongono
            if consumo + nuovo_consumo > 1.5:
                return False
    return True

# Funzione per trovare la migliore fascia oraria per un intervallo di utilizzo
def trova_migliore_fascia(start, durata, appliance, intervalli):
    best_start = None
    best_cost = float('inf')

    for fascia, orari in sorted(fasce_orarie.items(), key=lambda x: x[1]["costo"]):
        fascia_start = datetime.strptime(f"{date} {orari['start']}", "%Y-%m-%d %H:%M")
        fascia_end = datetime.strptime(f"{date} {orari['end']}", "%Y-%m-%d %H:%M")

        current_start = max(fascia_start, start - (start - fascia_start) % timedelta(minutes=1))
        while current_start + durata <= fascia_end:
            current_end = current_start + durata
            if verifica_consumo(intervalli, current_start, current_end, consumi[appliance]):
                cost = calcola_costo_intervallo(current_start, current_end, appliance)
                if cost < best_cost:
                    best_start = current_start
                    best_cost = cost
            current_start += timedelta(minutes=1)

    return best_start, best_cost

# Funzione per ottimizzare gli intervalli di utilizzo
def ottimizza_utilizzo(usage_list, appliance, intervalli):
    optimized_intervals = []
    total_cost = 0

    for start_time, end_time in usage_list:
        start = str_to_datetime(date, start_time)
        end = str_to_datetime(date, end_time)
        durata = end - start

        best_start, best_cost = trova_migliore_fascia(start, durata, appliance, intervalli)
        if best_start:
            best_end = best_start + durata
            optimized_intervals.append((best_start, best_end, consumi[appliance]))
            total_cost += best_cost

    return optimized_intervals, total_cost

# Dati di esempio
date = "2014-07-02"

usage_tumble_dryer = [
    ("11:05:34", "11:07:04"),
    ("11:21:04", "11:23:20"),
    ("16:09:05", "16:09:20")
]

usage_washing_machine = [
    ("02:22:20", "02:22:25"),
    ("12:18:34", "12:58:05"),
    ("13:00:49", "13:01:04"),
    ("13:08:49", "13:09:04")
]

usage_dishwasher = []  # Lavastoviglie non utilizzata

# Ottimizza gli intervalli di utilizzo rispettando il limite di consumo
intervalli_utilizzo = []
intervalli_tumble_dryer, costo_tumble_dryer_ottimizzato = ottimizza_utilizzo(usage_tumble_dryer, "asciugatrice", intervalli_utilizzo)
intervalli_utilizzo += intervalli_tumble_dryer

intervalli_washing_machine, costo_washing_machine_ottimizzato = ottimizza_utilizzo(usage_washing_machine, "lavatrice", intervalli_utilizzo)
intervalli_utilizzo += intervalli_washing_machine

print("Nuovi intervalli di utilizzo:")
for inizio, fine, consumo in intervalli_utilizzo:
    print(f"Inizio: {inizio.strftime('%H:%M:%S')}, Fine: {fine.strftime('%H:%M:%S')}, Consumo: {consumo} kW")

print(f"Costo ottimizzato totale Asciugatrice: €{costo_tumble_dryer_ottimizzato:.4f}")
print(f"Costo ottimizzato totale Lavatrice: €{costo_washing_machine_ottimizzato:.4f}")


Nuovi intervalli di utilizzo:
Inizio: 19:00:00, Fine: 19:01:30, Consumo: 1.2 kW
Inizio: 19:00:00, Fine: 19:02:16, Consumo: 1.2 kW
Inizio: 19:00:00, Fine: 19:00:15, Consumo: 1.2 kW
Inizio: 07:00:00, Fine: 07:00:05, Consumo: 1.0 kW
Inizio: 19:03:00, Fine: 19:42:31, Consumo: 1.0 kW
Inizio: 19:03:00, Fine: 19:03:15, Consumo: 1.0 kW
Inizio: 19:03:00, Fine: 19:03:15, Consumo: 1.0 kW
Costo ottimizzato totale Asciugatrice: €0.0120
Costo ottimizzato totale Lavatrice: €0.1003


In [ ]:
from datetime import datetime, timedelta

# Definiamo le fasce orarie con i relativi costi
fasce_orarie = {
    "F1": {"start": "08:00", "end": "19:00", "costo": 0.25},
    "F2": {"start": "07:00", "end": "08:00", "costo": 0.15},
    "F2_b": {"start": "19:00", "end": "23:00", "costo": 0.15},
    "F3": {"start": "23:00", "end": "07:00", "costo": 0.10}
}

# Definiamo i consumi degli elettrodomestici in kW
consumi = {
    "lavatrice": 1.0,
    "asciugatrice": 1.2,
    "lavastoviglie": 1.0
}

# Funzione per convertire stringhe di tempo in oggetti datetime
def str_to_datetime(date_str, time_str):
    return datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

# Funzione per calcolare la durata in ore tra due datetime
def calcola_durata(start, end):
    return (end - start).seconds / 3600

# Funzione per determinare la fascia oraria di un timestamp
def determina_fascia(timestamp):
    time = timestamp.time()
    for fascia, orari in fasce_orarie.items():
        start = datetime.strptime(orari["start"], "%H:%M").time()
        end = datetime.strptime(orari["end"], "%H:%M").time()
        if start <= time < end or (start > end and (time >= start or time < end)):
            return fascia
    return None

# Funzione per calcolare il costo di utilizzo di un intervallo
def calcola_costo_intervallo(start, end, appliance):
    durata = calcola_durata(start, end)
    fascia = determina_fascia(start)
    costo = durata * fasce_orarie[fascia]["costo"] * consumi[appliance]
    return costo

# Funzione per verificare se un nuovo intervallo può essere aggiunto senza superare il limite di consumo
def verifica_consumo(intervalli, nuovo_inizio, nuovo_fine, nuovo_consumo):
    for inizio, fine, consumo in intervalli:
        if (inizio < nuovo_fine and nuovo_inizio < fine):  # Intervalli si sovrappongono
            if consumo + nuovo_consumo > 1.5:
                return False
    return True

# Funzione per verificare sovrapposizioni di intervalli dello stesso appliance
def overlapping_intervals(intervalli, nuovo_inizio, nuovo_fine, appliance):
    for inizio, fine, _appliance in intervalli:
        if _appliance == appliance and (inizio < nuovo_fine and nuovo_inizio < fine):
            return True
    return False

# Funzione per trovare la migliore fascia oraria per un intervallo di utilizzo
def trova_migliore_fascia(start, durata, appliance, intervalli):
    best_start = None
    best_cost = float('inf')

    for fascia, orari in sorted(fasce_orarie.items(), key=lambda x: x[1]["costo"]):
        fascia_start = datetime.strptime(f"{date} {orari['start']}", "%Y-%m-%d %H:%M")
        fascia_end = datetime.strptime(f"{date} {orari['end']}", "%Y-%m-%d %H:%M")

        current_start = max(fascia_start, start - (start - fascia_start) % timedelta(minutes=1))
        while current_start + durata <= fascia_end:
            current_end = current_start + durata
            if verifica_consumo(intervalli, current_start, current_end, consumi[appliance]) \
                    and not overlapping_intervals(intervalli, current_start, current_end, appliance):
                cost = calcola_costo_intervallo(current_start, current_end, appliance)
                if cost < best_cost:
                    best_start = current_start
                    best_cost = cost
            current_start += timedelta(minutes=1)

    return best_start, best_cost

# Funzione per ottimizzare gli intervalli di utilizzo
def ottimizza_utilizzo(usage_list, appliance):
    optimized_intervals = []
    total_cost = 0

    for start_time, end_time in usage_list:
        start = str_to_datetime(date, start_time)
        end = str_to_datetime(date, end_time)
        durata = end - start

        best_start, best_cost = trova_migliore_fascia(start, durata, appliance, optimized_intervals)
        if best_start:
            best_end = best_start + durata
            optimized_intervals.append((best_start, best_end, consumi[appliance]))
            total_cost += best_cost

    return optimized_intervals, total_cost

# Dati di esempio
date = "2014-07-02"

usage_tumble_dryer = [
    ("11:05:34", "11:07:04"),
    ("11:21:04", "11:23:20"),
    ("16:09:05", "16:09:20")
]

usage_washing_machine = [
    ("02:22:20", "02:22:25"),
    ("12:18:34", "12:58:05"),
    ("13:00:49", "13:01:04"),
    ("13:08:49", "13:09:04")
]

usage_dishwasher = []  # Lavastoviglie non utilizzata

# Ottimizza gli intervalli di utilizzo rispettando il limite di consumo
intervalli_utilizzo = []
intervalli_tumble_dryer, costo_tumble_dryer_ottimizzato = ottimizza_utilizzo(usage_tumble_dryer, "asciugatrice")
intervalli_utilizzo += intervalli_tumble_dryer

intervalli_washing_machine, costo_washing_machine_ottimizzato = ottimizza_utilizzo(usage_washing_machine, "lavatrice")
intervalli_utilizzo += intervalli_washing_machine

print("Nuovi intervalli di utilizzo:")
for inizio, fine, consumo in intervalli_utilizzo:
    print(f"Inizio: {inizio.strftime('%H:%M:%S')}, Fine: {fine.strftime('%H:%M:%S')}, Consumo: {consumo} kW")

print(f"Costo ottimizzato totale Asciugatrice: €{costo_tumble_dryer_ottimizzato:.4f}")
print(f"Costo ottimizzato totale Lavatrice: €{costo_washing_machine_ottimizzato:.4f}")


Nuovi intervalli di utilizzo:
Inizio: 19:00:00, Fine: 19:01:30, Consumo: 1.2 kW
Inizio: 19:02:00, Fine: 19:04:16, Consumo: 1.2 kW
Inizio: 19:05:00, Fine: 19:05:15, Consumo: 1.2 kW
Inizio: 07:00:00, Fine: 07:00:05, Consumo: 1.0 kW
Inizio: 19:00:00, Fine: 19:39:31, Consumo: 1.0 kW
Inizio: 19:40:00, Fine: 19:40:15, Consumo: 1.0 kW
Inizio: 19:41:00, Fine: 19:41:15, Consumo: 1.0 kW
Costo ottimizzato totale Asciugatrice: €0.0120
Costo ottimizzato totale Lavatrice: €0.1003
